In [4]:
# Refactoring NASEM model
import nasem_dairy as nd
import pandas as pd
import math
from pprint import pprint


# use package dir to always use /data folder regardless of where this is executed from
import importlib_resources

path_to_package_data = importlib_resources.files("nasem_dairy.data")


# Read_csv to load required data into env
user_diet_in, animal_input_in, equation_selection_in = nd.read_csv_input(path_to_package_data.joinpath("./input.csv"))

# Load feed library
feed_library_in = pd.read_csv(path_to_package_data.joinpath("NASEM_feed_library.csv"))

# Prepare infusion data (This will be optional as a default dict of 0 is provided to function otherwise)
infusion_custom = nd.read_infusion_input(path_to_package_data.joinpath("infusion_input.csv"))

In [11]:
def dev_run_NASEM_model(user_diet: pd.DataFrame, 
                        animal_input: dict, 
                        equation_selection: dict, 
                        feed_library_df: pd.DataFrame, 
                        coeff_dict: dict = nd.coeff_dict,
                        infusion_input: dict = nd.infusion_dict
                        ):
    """
    Run the NASEM (National Academies of Sciences, Engineering, and Medicine) Nutrient Requirements of Dairy Cattle  model.

    Parameters
    ----------
    user_diet : pd.DataFrame
        DataFrame containing user-defined diet composition.
    animal_input : dict
        Dictionary containing animal-specific input values.
    equation_selection : dict
        Dictionary containing equation selection criteria.
    feed_library_df : pd.DataFrame
        DataFrame containing the feed library data.
    coeff_dict : dict, optional
        Dictionary containing coefficients for the model, by default `nd.coeff_dict`.
    infusion_input : dict, optional
        Dictionary containing infusion input data, by default `nd.infusion_dict`.

    Returns
    -------
    Multiple
        Currently returns animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict
        To be updated

    Notes
    -----
    - 

    Examples
    --------
    Run the NASEM dairy model with user-defined inputs:

    ```python
    import nasem_dairy as nd
    nd.dev_run_NASEM_model(
        user_diet=user_diet_df,
        animal_input=animal_input_dict,
        equation_selection=equation_selection_dict,
        feed_library_df=feed_library_df
    )
    ```
    """
    ########################################
    # Step 1: Read User Input
    ########################################
    # prevent mutable changes outside of expected scope (especially for Shiny):
    user_diet = user_diet.copy()
    animal_input = animal_input.copy()

    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = user_diet['Feedstuff'].tolist()
    # feed_data = fl_get_rows(list_of_feeds, path_to_db)
    feed_data = nd.fl_get_feed_rows(list_of_feeds, feed_library_df)


    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    # list_of_feeds = user_diet['Feedstuff'].tolist()
    # feed_data = nd.fl_get_rows(list_of_feeds, '../../src/nasem_dairy/data/diet_database.db')

    feed_data = feed_data.reset_index(names='Feedstuff')

    diet_info_initial = pd.DataFrame({'Feedstuff': user_diet['Feedstuff']})
    diet_info_initial = diet_info_initial.merge(feed_data, how='left', on='Feedstuff')

    # Add Fd_DMInp to the diet_info dataframe
    Fd_DMInp = user_diet.set_index('Feedstuff')['kg_user'] / user_diet['kg_user'].sum()

    diet_info_initial.insert(1, 'Fd_DMInp', 
        Fd_DMInp.reindex(
            diet_info_initial['Feedstuff']).values)

    diet_info_initial['Fd_DMIn'] = diet_info_initial['Fd_DMInp'] * \
        animal_input['DMI']      # Should this be done after DMI equations?

    # Add Fd_DNDF48 column, need to add to the database
    # diet_info_initial['Fd_DNDF48'] = 0

    # Calculate additional physiology values
    animal_input['An_PrePartDay'] = animal_input['An_GestDay'] - animal_input['An_GestLength']
    animal_input['An_PrePartWk'] = animal_input['An_PrePartDay'] / 7

    del (list_of_feeds, Fd_DMInp)

    # Check equation_selection to make sure they are integers.
    # This is especially important for Shiny, which may return strings
    # It's important they are correct for if statements below.
    equation_selection_in = equation_selection.copy()
    equation_selection = {}

    for key, value in equation_selection_in.items():
        try:
            num_value = int(value)
            equation_selection[key] = num_value
        except ValueError:
            print(f"Unable to convert '{value}' to an integer for key '{key}'")
    
    del(equation_selection_in)

    # if animal_input['An_StatePhys'] != 'Lactating Cow':
    #     animal_input['Trg_MilkProd'] = None

    ############################################################################################
    # Start of refactored NASEM 
    ############################################################################################
            
 
    # # Create infusion_input
    # if equation_selection['use_infusions'] == 0:
    #     # If no infusions then all values are set to 0
    #     no_infusion_input = nd.infusion_dict
    # elif equation_selection['use_infusions'] == 1:
    #     infusion_input = infusion_data
    # else:
    #     raise ValueError(
    #         f"Invalid use_infusions: {equation_selection['use_infusions']} was entered. Must be 0 or 1")

    ########################################
    # Step 1.5: Input Based Equations
    ########################################
    K_FeCPend_ClfLiq = nd.calculate_K_FeCPend_ClfLiq(animal_input['An_StatePhys'],
                                                     equation_selection['NonMilkCP_ClfLiq'])

    ########################################
    # Step 2: DMI Equations
    ########################################
    # Calculate Target milk net energy
    Trg_NEmilk_Milk = nd.calculate_Trg_NEmilk_Milk(animal_input['Trg_MilkTPp'],
                                                animal_input['Trg_MilkFatp'],
                                                animal_input['Trg_MilkLacp'])
    
    # # Need to precalculate Dt_NDF for DMI predicitons, this will be based on the user entered DMI (animal_input['DMI])
    Dt_NDF = (diet_info_initial['Fd_NDF'] * diet_info_initial['Fd_DMInp']).sum()

    if equation_selection['DMIn_eqn'] == 0:
        # print('Using user input DMI')
        pass

    # Predict DMI for lactating cow
    elif equation_selection['DMIn_eqn'] == 8:
        # print("using DMIn_eqn: 8")
        animal_input['DMI'] = nd.calculate_Dt_DMIn_Lact1(
            
            animal_input['Trg_MilkProd'],
            animal_input['An_BW'],
            animal_input['An_BCS'],
            animal_input['An_LactDay'],
            animal_input['An_Parity_rl'],
            Trg_NEmilk_Milk
            )

    # Predict DMI for heifers
    elif equation_selection['DMIn_eqn'] in [2, 3, 4, 5, 6, 7, 12, 13, 14, 15, 16, 17]:
        animal_input['DMI'] = nd.heifer_growth(
            equation_selection['DMIn_eqn'],
            # diet_info.loc['Diet', 'Fd_NDF'],
            Dt_NDF,
            animal_input['An_BW'],
            animal_input['An_BW_mature'],
            animal_input['An_PrePartWk'],
            coeff_dict)

    elif equation_selection['DMIn_eqn'] in [10, 11]:
        animal_input['DMI'] = nd.dry_cow_equations(
            equation_selection['DMIn_eqn'],
            animal_input['An_BW'],
            animal_input['An_PrePartWk'],
            animal_input['An_GestDay'],
            animal_input['An_GestLength'],
            Dt_NDF,
            coeff_dict)

    else:
        # It needs to catch all possible solutions, otherwise it's possible that it stays unchanged without warning
        print("DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: " + str(
            equation_selection['DMIn_eqn']))

    # Calculated again as part of diet_data, value may change depending on DMIn_eqn selections
    del (Dt_NDF)

    ########################################
    # Step 3: Feed Based Calculations
    ########################################
    # Calculate An_DMIn_BW with the final DMI value; required for calculating diet_data_initial
    An_DMIn_BW = nd.calculate_An_DMIn_BW(animal_input['An_BW'],
                                         animal_input['DMI'])
    Fe_rOMend = nd.calculate_Fe_rOMend(animal_input['DMI'],
                                       coeff_dict)

    diet_info = nd.calculate_diet_info(animal_input['DMI'],
                                       animal_input['An_StatePhys'],
                                       equation_selection['Use_DNDF_IV'],
                                       diet_info=diet_info_initial,
                                       coeff_dict=coeff_dict)
    # All equations in the f dataframe go into calculate_diet_info()
    # This includes micronutrient calculations which are no longer handled by seperate functions

    diet_data_initial = nd.calculate_diet_data_initial(diet_info,
                                                       animal_input['DMI'],
                                                       animal_input['An_BW'],
                                                       animal_input['An_StatePhys'],
                                                       An_DMIn_BW,
                                                       Fe_rOMend,
                                                       coeff_dict)
    # diet_data contains everything starting with "Dt_"

    ########################################
    # Step 4: Infusion Calculations
    ########################################

    infusion_data = nd.calculate_infusion_data(infusion_input, animal_input['DMI'], coeff_dict)


    ########################################
    # Step 5: Animal Level Calculations
    ########################################
    # Combine Diet and Infusion nutrient supplies
    An_data_initial = nd.calculate_An_data_initial(animal_input,
                                                   diet_data_initial,
                                                   infusion_data,
                                                   coeff_dict)

    ########################################
    # Step 6: Rumen Digestion Calculations
    ########################################
    # Rumen Digestability Coefficients
    Rum_dcNDF = nd.calculate_Rum_dcNDF(animal_input['DMI'],
                                       diet_data_initial['Dt_NDFIn'],
                                       diet_data_initial['Dt_StIn'],
                                       diet_data_initial['Dt_CPIn'],
                                       diet_data_initial['Dt_ADFIn'],
                                       diet_data_initial['Dt_ForWet'])

    Rum_dcSt = nd.calculate_Rum_dcSt(animal_input['DMI'],
                                     diet_data_initial['Dt_ForNDF'],
                                     diet_data_initial['Dt_StIn'],
                                     diet_data_initial['Dt_ForWet'])

    # Rumen Digestable Intakes
    Rum_DigNDFIn = nd.calculate_Rum_DigNDFIn(Rum_dcNDF,
                                             diet_data_initial['Dt_NDFIn'])

    Rum_DigStIn = nd.calculate_Rum_DigStIn(Rum_dcSt,
                                           diet_data_initial['Dt_StIn'])

    ########################################
    # Step 7: Microbial Protein Calculations
    ########################################
    if equation_selection['MiN_eqn'] == 1:
        RDPIn_MiNmax = nd.calculate_RDPIn_MiNmax(animal_input['DMI'],
                                                 An_data_initial['An_RDP'],
                                                 An_data_initial['An_RDPIn'])
        MiN_Vm = nd.calculate_MiN_Vm(RDPIn_MiNmax,
                                     coeff_dict)
        Du_MiN_g = nd.calculate_Du_MiN_NRC2021_g(MiN_Vm,
                                                 Rum_DigNDFIn,
                                                 Rum_DigStIn,
                                                 An_data_initial['An_RDPIn_g'],
                                                 coeff_dict)
    elif equation_selection['MiN_eqn'] == 2:
        Du_MiN_g = nd.calculate_Du_MiN_VTln_g(diet_data_initial['Dt_rOMIn'],
                                              diet_data_initial['Dt_ForNDFIn'],
                                              An_data_initial['An_RDPIn'],
                                              Rum_DigStIn,
                                              Rum_DigNDFIn,
                                              coeff_dict)
    elif equation_selection['MiN_eqn'] == 3:
        Du_MiN_g = nd.calculate_Du_MiN_VTnln_g(An_data_initial['An_RDPIn'],
                                               Rum_DigNDFIn,
                                               Rum_DigStIn)
    else:
        raise ValueError(
            f"Invalid MiN_eqn: {equation_selection['MiN_eqn']} was entered. Must choose 1, 2 or 3.")

    Du_MiCP_g = nd.calculate_Du_MiCP_g(Du_MiN_g)
    Du_MiTP_g = nd.calculate_Du_MiTP_g(Du_MiCP_g, coeff_dict)
    Du_MiCP = nd.calculate_Du_MiCP(Du_MiCP_g)
    Du_idMiCP_g = nd.calculate_Du_idMiCP_g(Du_MiCP_g,
                                           coeff_dict)
    Du_idMiCP = nd.calculate_Du_idMiCP(Du_idMiCP_g)

    ########################################
    # Step 7.1: Fe_CP Calculation /  Finish Dt_ and An_ calculations
    ########################################
    # Required to finish Dt_ and An_ calculations
    Fe_RUP = nd.calculate_Fe_RUP(An_data_initial['An_RUPIn'],
                                 infusion_data['InfSI_TPIn'],
                                 An_data_initial['An_idRUPIn'])
    Fe_RumMiCP = nd.calculate_Fe_RumMiCP(Du_MiCP,
                                         Du_idMiCP)
    Fe_CPend_g = nd.calculate_Fe_CPend_g(animal_input['An_StatePhys'],
                                         An_data_initial['An_DMIn'],
                                         An_data_initial['An_NDF'],
                                         animal_input['DMI'],
                                         diet_data_initial['Dt_DMIn_ClfLiq'],
                                         K_FeCPend_ClfLiq)
    Fe_CPend = nd.calculate_Fe_CPend(Fe_CPend_g)
    Fe_CP = nd.calculate_Fe_CP(animal_input['An_StatePhys'],
                               diet_data_initial['Dt_CPIn_ClfLiq'],
                               diet_data_initial['Dt_dcCP_ClfDry'],
                               An_data_initial['An_CPIn'],
                               Fe_RUP,
                               Fe_RumMiCP,
                               Fe_CPend,
                               infusion_data['InfSI_NPNCPIn'],
                               coeff_dict)

    ########################################
    # Step 7.2: Complete diet_data and An_data
    ########################################
    diet_data = nd.calculate_diet_data_complete(diet_data_initial,
                                                animal_input['An_StatePhys'],
                                                Fe_CP,
                                                equation_selection,
                                                coeff_dict)
    An_data = nd.calculate_An_data_complete(An_data_initial,
                                            diet_data,
                                            animal_input['An_StatePhys'],
                                            Fe_CP,
                                            infusion_data,
                                            equation_selection,
                                            coeff_dict)

    ########################################
    # Step 8: Amino Acid Calculations
    ########################################
    AA_list = ['Arg', 'His', 'Ile', 'Leu',
               'Lys', 'Met', 'Phe', 'Thr', 'Trp', 'Val']
    AA_values = pd.DataFrame(index=AA_list)
    # Dataframe for storing all individual amino acid values

    AA_values['Du_AAMic'] = nd.calculate_Du_AAMic(Du_MiTP_g,
                                                  AA_list,
                                                  coeff_dict)
    AA_values['Du_IdAAMic'] = nd.calculate_Du_IdAAMic(AA_values['Du_AAMic'],
                                                      coeff_dict)
    AA_values['Abs_AA_g'] = nd.calculate_Abs_AA_g(diet_data,
                                                  AA_values['Du_IdAAMic'],
                                                  AA_list)
    AA_values['mPrtmx_AA'] = nd.calculate_mPrtmx_AA(AA_list,
                                                    coeff_dict)
    f_mPrt_max = nd.calculate_f_mPrt_max(animal_input['An_305RHA_MlkTP'],
                                         coeff_dict)
    AA_values['mPrtmx_AA2'] = nd.calculate_mPrtmx_AA2(AA_values['mPrtmx_AA'],
                                                      f_mPrt_max)
    AA_values['AA_mPrtmx'] = nd.calculate_AA_mPrtmx(AA_list,
                                                    coeff_dict)
    AA_values['mPrt_AA_01'] = nd.calculate_mPrt_AA_01(AA_values['AA_mPrtmx'],
                                                      AA_list,
                                                      coeff_dict)
    AA_values['mPrt_k_AA'] = nd.calculate_mPrt_k_AA(AA_values['mPrtmx_AA2'],
                                                    AA_values['mPrt_AA_01'],
                                                    AA_values['AA_mPrtmx'])

    ########################################
    # Step 9: Uterine Bodyweight Calculations
    ########################################
    Uter_Wtpart = nd.calculate_Uter_Wtpart(
        animal_input['Fet_BWbrth'], coeff_dict)
    Uter_Wt = nd.calculate_Uter_Wt(animal_input['An_Parity_rl'],
                                   animal_input['An_AgeDay'],
                                   animal_input['An_LactDay'],
                                   animal_input['An_GestDay'],
                                   animal_input['An_GestLength'],
                                   Uter_Wtpart,
                                   coeff_dict)
    GrUter_Wtpart = nd.calculate_GrUter_Wtpart(
        animal_input['Fet_BWbrth'], coeff_dict)
    GrUter_Wt = nd.calculate_GrUter_Wt(animal_input['An_GestDay'],
                                       animal_input['An_GestLength'],
                                       Uter_Wt,
                                       GrUter_Wtpart,
                                       coeff_dict)
    Uter_BWgain = nd.calculate_Uter_BWgain(animal_input['An_LactDay'],
                                           animal_input['An_GestDay'],
                                           animal_input['An_GestLength'],
                                           Uter_Wt,
                                           coeff_dict)
    GrUter_BWgain = nd.calculate_GrUter_BWgain(animal_input['An_LactDay'],
                                               animal_input['An_GestDay'],
                                               animal_input['An_GestLength'],
                                               GrUter_Wt,
                                               Uter_BWgain,
                                               coeff_dict)

    ########################################
    # Step 10: Metabolizable Protein Intake
    ########################################
    Du_idMiTP_g = nd.calculate_Du_idMiTP_g(Du_idMiCP_g,
                                           coeff_dict)
    Du_idMiTP = nd.calculate_Du_idMiTP(Du_idMiTP_g)
    An_MPIn = nd.calculate_An_MPIn(diet_data['Dt_idRUPIn'],
                                   Du_idMiTP)
    An_MPIn_g = nd.calculate_An_MPIn_g(An_MPIn)

    ########################################
    # Temporary results export
    ########################################
    
    model_out_dict = {
        'infusion_data' : infusion_data
    }

    return animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict


# ### RUN MODEL ###
# animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data = run_NASEM()

# Run model - different options

In [ ]:
pd.set_option('display.max_columns', 100)

In [5]:
animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

# diet_info
# An_data


## Toggle infusions
Notes: found that infusions are being imported within the function, need to move all file imports out of the run model function.
There is also double up of the logic for use_infusions, so modifying that to check which one to use once. 
It could actually be avoided by just providing a default infusion dictionary if one is not given to function directly.
These changes make the 'use_infusions' flag obsolete

In [ ]:
# Test DMI equations
equation_selection_in['use_infusions'] = 0 # FALSE

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

# diet_info
# infusion_data


In [ ]:
# equation_selection_in['use_infusions'] = 1

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict,
    infusion_input= infusion_custom)

# diet_info
model_out_dict

## Test DMI equation selections

In [ ]:
# 0
# 8
# [2, 3, 4, 5, 6, 7, 12, 13, 14, 15, 16, 17]

In [8]:
# Equation = 0 should match user input:
equation_selection_in['DMIn_eqn'] = 0
animal_input_in['DMI'] = 24.521 # same as .csv file

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

animal_input['DMI']

24.521

In [12]:
# Equation = 8 should calculate calculate_Dt_DMIn_Lact1()
equation_selection_in['DMIn_eqn'] = 8
animal_input_in['DMI'] = 24.521 # same as .csv file

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

animal_input['DMI']

21.687577670699824